# IMPORT THE LIBRARIES

In [1]:
#exc1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
from datetime import timedelta
import re

# LOAD THE PI DATA

In [52]:
#Sheet From Pi
#Step1: Load the Data for the month
df_2020Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\PressureFilter_perCycle_param_042020.xlsx', sheet_name='PiData_PressureFilter', index_col=False)

#Drop the row[1]
df_2020Pi = df_2020Pi.drop(df_2020Pi.index[0])

#Step2: Parse the DATETIME column
df_2020Pi['DATETIME'] = df_2020Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_2020Pi.set_index('DATETIME', inplace=True)
df_comb = df_2020Pi

for i in df_comb.columns:
    df_comb[i] = pd.to_numeric(df_comb[i], errors='coerce')


# DECLARE THE FUNCTIONS

In [53]:
#---fcn defn--start
#This function is used to replace the 'No good/ not enough data for calculation' value in the dataframe for data from PiData

def det_no_data(value):
    if type(value) == str:
        reg = re.search(r'\bCalculation\b', value)
        return reg[0]
    else:
        return value
#------fcn defn end

#fcn definition        
#used in the laboratory analysis results data


def remove_no_good_data_N2Dry_Feed(df): #removes the string value and fills it with the last good value
    
    df = pd.to_numeric(df, errors='coerce')
    for x in range(0, len(df)):
        if type(df[x]) == str:
            df[x] = df[x-1]
    df.fillna(method='ffill', inplace=True)
    return df

#---------------------------
def replace_no_good_data_with_LastGoodValues(df_col):
    
    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]== str):
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = np.nan

    df_col.fillna(method='ffill', inplace= True)
    return df_col
#--------------------------------------------
def replace_no_good_data_with_Zero(df_col):

    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]) == str:
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = 0
    df_col.fillna(value=0, inplace=True)
    
    return df_col
#--------------------------------------------
#FILTRATION TIME function
#--------------------------------------------
def generate_DF_comm(df_coltitle, str_colname):
        
    A_Filt_MaxList= list()
    A_Filt_MaxList[:] = []
    A_feedt_Val_List = list()
    A_feedt_Val_List[:] = []
    A_feedt_index_List = list()
    A_feedt_index_List[:] = []
    A_Pair_Filt_List = list()
    A_Pair_Filt_List[:] = []
    try:         
        for i in range(0, len(keyslist)):
            A_feedt_Val_List = list()
            A_feedt_Val_List[:] = []
            for j in range(keyslist[i], ft_cyc_lim[keyslist[i]]):
                A_feedt = df_coltitle[j]
                A_feedt_Val_List.append(A_feedt)
            A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
            A_feedt_index_List.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))  
        A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
    except:
        pass
    df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
    df_A_Pair_Filt_t.columns = [str_colname, 'DATETIME']
    df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)  
    return df_A_Pair_Filt_t
#--------------------------------------------
#finding peaks FT for A_filtration time
#---function start---------
# the ft_cyc_lim is the index for filtration start and n2 drying end as a pair
def getTheLocalMinMax(df_col1, df_col2):
    df_col1 = pd.to_numeric(df_col1, errors='coerce')
    b = (np.diff(np.sign(np.diff(df_col1))) > 0).nonzero()[0] + 1         # local min
   
    df_col2 = pd.to_numeric(df_col2, errors='coerce')
    f = (np.diff(np.sign(np.diff(df_col2))) < 0).nonzero()[0] + 1         # local max

    try:
        ft_cyc_lim = {}
        for i in range(0, len(b)):
            for j in range(0, len(f)):
                if f[j] > b[i] and f[j] < b[i+1]:
                    if f[j+1]<b[i+1]:
                        cyc_start = b[i]
                        cyc_end = f[j+1]
                        ft_cyc_lim.setdefault(cyc_start,cyc_end)
    except IndexError:
        print('\nDone')
    return ft_cyc_lim

#end of function----------------
#code for getting the CWP for CakeWash1 and CakeWash2

#----------------------------------------------------
def generate_DF_CW1(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslist)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))

            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
            A_CWP_1_inner.append(A_CWP_1)

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------

#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_DF_CW2(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslist)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslist[i],ft_cyc_lim[keyslist[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslist[i]:ft_cyc_lim[keyslist[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]])))
            print(df_comb.index[keyslist[i]])
            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
            A_CWP_1_inner.append(A_CWP_1)
            

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------
#---function to generate N2DRY DF
def generate_DF_for_N2DRY(df_colN2, str_colname):
    
    A_Pair_N2_Dry_List = list()
    A_Pair_N2_Dry_List[:] = []
    A_t_N2_Dry_indexList = list()
    A_t_N2_Dry_indexList[:] = []
    A_t_N2_Dry_val = list()
    A_t_N2_Dry_val[:] = []
    
    try:         
        for i in range(0, len(keyslist)):   
            A_t_N2_Dry = df_colN2[ft_cyc_lim[keyslist[i]]]
            A_t_N2_Dry_val.append(A_t_N2_Dry)
            A_t_N2_Dry_indexList.append((str(df_comb.index[keyslist[i]])+" to "+str(df_comb.index[ft_cyc_lim[keyslist[i]]]))) 
    except:
        pass
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))
    df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
    df_A_Pair_N2DRY.columns = [str_colname, 'DATETIME']
    df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_N2DRY

#------------------------------------------------
#Functions for BRAVO FILTER Parameters

def generate_dfB_comm(df_coltitle, str_colname):
        
    A_Filt_MaxList= list()
    A_Filt_MaxList[:] = []
    A_feedt_Val_List = list()
    A_feedt_Val_List[:] = []
    A_feedt_index_List = list()
    A_feedt_index_List[:] = []
    A_Pair_Filt_List = list()
    A_Pair_Filt_List[:] = []
    try:         
        for i in range(0, len(keyslistB)):
            A_feedt_Val_List = list()
            A_feedt_Val_List[:] = []
            for j in range(keyslistB[i], ft_cyc_limB[keyslistB[i]]):
                A_feedt = df_coltitle[j]
                A_feedt_Val_List.append(A_feedt)
            A_Filt_MaxList.append(sorted(A_feedt_Val_List)[-1])
            A_feedt_index_List.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))  
        A_Pair_Filt_List = list(zip(A_Filt_MaxList, A_feedt_index_List[0:]))      
    except:
        pass
    df_A_Pair_Filt_t = pd.DataFrame(A_Pair_Filt_List)
    df_A_Pair_Filt_t.columns = [str_colname, 'DATETIME']
    df_A_Pair_Filt_t.set_index('DATETIME', inplace=True)  
    return df_A_Pair_Filt_t
#--------------------------------------------


#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_dfB_CW1(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslistB)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslistB[i],ft_cyc_limB[keyslistB[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslistB[i]:ft_cyc_limB[keyslistB[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))
            print(df_comb.index[keyslistB[i]])
            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-1]
            A_CWP_1_inner.append(A_CWP_1)               
            
    except IndexError:
        print('IndexError')
        
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------

#code for getting the CWP for CakeWash1 and CakeWash2
#----------------------------------------------------
def generate_dfB_CW2(df_col, str_colname):
    A_CWP_1_indexList = list()
    A_CWP_1_indexList[:] = []
    A_CWP_1_inner = list()
    A_CWP_1_inner[:] = []
    A_Pair_CWP_1_List = list()
    try:
        for i in range(0, len(keyslistB)):
            A_CWPList_i = list()
            A_CWPList_i[:] = []
            d = list()
            d[:] = []

            A_CWPList_i = list(np.arange(keyslistB[i],ft_cyc_limB[keyslistB[i]]))
            d = (np.diff(np.sign(np.diff(df_col.iloc[keyslistB[i]:ft_cyc_limB[keyslistB[i]]])))< 0).nonzero()[0] + 1
            A_CWP_for_sort = list()
            A_CWP_for_sort[:] = [] 
            A_CWP_1_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]])))
            print(df_comb.index[keyslistB[i]])
            for i in range(0, len(d)):
                CWP_val_shortlist = df_col.iloc[A_CWPList_i[d[i]]]
                A_CWP_for_sort.append(CWP_val_shortlist)
            A_CWP_1 = sorted(A_CWP_for_sort)[len(A_CWP_for_sort)-2]
            A_CWP_1_inner.append(A_CWP_1)               

    except IndexError:
        print('IndexError')
    A_Pair_CWP_1_List = list(zip(A_CWP_1_inner, A_CWP_1_indexList[0:]))
    df_A_Pair_CWP_1 = pd.DataFrame(A_Pair_CWP_1_List)
    df_A_Pair_CWP_1.columns = [str_colname, 'DATETIME']
    df_A_Pair_CWP_1.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_CWP_1
#----------------------------------------------------
#---function to generate N2DRY DF
def generate_dfB_for_N2DRY(df_colN2, str_colname):
    
    A_Pair_N2_Dry_List = list()
    A_Pair_N2_Dry_List[:] = []
    A_t_N2_Dry_indexList = list()
    A_t_N2_Dry_indexList[:] = []
    A_t_N2_Dry_val = list()
    A_t_N2_Dry_val[:] = []
    
    try:         
        for i in range(0, len(keyslistB)):   
            A_t_N2_Dry = df_colN2[ft_cyc_limB[keyslistB[i]]]
            A_t_N2_Dry_val.append(A_t_N2_Dry)
            A_t_N2_Dry_indexList.append((str(df_comb.index[keyslistB[i]])+" to "+str(df_comb.index[ft_cyc_limB[keyslistB[i]]]))) 
    except:
        pass
    
    A_Pair_N2_Dry_List = list(zip(A_t_N2_Dry_val, A_t_N2_Dry_indexList[0:]))
    df_A_Pair_N2DRY = pd.DataFrame(A_Pair_N2_Dry_List)
    df_A_Pair_N2DRY.columns = [str_colname, 'DATETIME']
    df_A_Pair_N2DRY.set_index('DATETIME', inplace=True)
    
    return df_A_Pair_N2DRY

#------------------------------------------------

print('all functions declared', datetime.datetime.now())

all functions declared 2020-07-29 18:48:39.375998


In [54]:
#Remove the no good data for t_FEED and N2_DRY ------------------

df_comb['A_t_FEED'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_FEED'])
df_comb['A_t_DRY'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_DRY'])

#Clean data from no good values
df_comb['A_t_CWSH1'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_CWSH1'])
df_comb['A_t_CWSH2'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_CWSH2'])
df_comb['A_t_PRESS1'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_PRESS1'])
df_comb['A_t_PRESS2'] = replace_no_good_data_with_LastGoodValues(df_comb['A_t_PRESS2'])

#Clean data from no good values
df_comb['106FT02A_MANFP'] = replace_no_good_data_with_Zero(df_comb['106FT02A_MANFP'])
df_comb['106FT02A_CWP'] = replace_no_good_data_with_Zero(df_comb['106FT02A_CWP'])
df_comb['106PU26A_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU26A_CUR'])
df_comb['106PU27A_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU27A_CUR'])
df_comb['PU26A_OP'] = replace_no_good_data_with_Zero(df_comb['PU26A_OP'])
df_comb['PU27A_OP'] = replace_no_good_data_with_Zero(df_comb['PU27A_OP'])

#Remove the no good data for t_FEED and N2_DRY for B dataframe------------------

df_comb['B_t_FEED'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_FEED'])
df_comb['B_t_DRY'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_DRY'])

#Clean data from no good values
df_comb['B_t_CWSH1'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_CWSH1'])
df_comb['B_t_CWSH2'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_CWSH2'])
df_comb['B_t_PRESS1'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_PRESS1'])
df_comb['B_t_PRESS2'] = replace_no_good_data_with_LastGoodValues(df_comb['B_t_PRESS2'])

#Clean data from no good values
df_comb['FT02B_MANFP'] = replace_no_good_data_with_Zero(df_comb['FT02B_MANFP'])
df_comb['FT02B_CWP'] = replace_no_good_data_with_Zero(df_comb['FT02B_CWP'])
df_comb['106PU26B_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU26B_CUR'])
df_comb['106PU27B_CUR'] = replace_no_good_data_with_Zero(df_comb['106PU27B_CUR'])
df_comb['PU26B_OP'] = replace_no_good_data_with_Zero(df_comb['PU26B_OP'])
df_comb['PU27B_OP'] = replace_no_good_data_with_Zero(df_comb['PU27B_OP'])

print('Done. All param_df_generated', datetime.datetime.now())
#------------------------------------------------

Done. All param_df_generated 2020-07-29 18:49:11.880256


# SELECT THE ALPHA PARAMETERS

In [55]:
#Get the local min, max. Then generate the keys
ft_cyc_lim = getTheLocalMinMax(df_comb['A_t_FEED'], df_comb['A_t_DRY'])
keyslist = list(ft_cyc_lim.keys())
#----
print('Done. Filtration Cycle Time generated.', datetime.datetime.now())

#Generate the dataframes
df_A_CWP_1 = generate_DF_CW1(df_comb['106FT02A_CWP'], 'A_CWP_1')
df_A_CWP_2 = generate_DF_CW2(df_comb['106FT02A_CWP'], 'A_CWP_2')
df_A_N2DRY = generate_DF_for_N2DRY(df_comb['A_t_DRY'], 'A_N2DRY')
df_A_MANFP = generate_DF_comm(df_comb['106FT02A_MANFP'], 'A_MANF_P')
df_A_t_FEED = generate_DF_comm(df_comb['A_t_FEED'], 'A_FILT_T')
df_A_t_CWSH1 = generate_DF_comm(df_comb['A_t_CWSH1'], 'A_CWSH1')
df_A_t_CWSH2 = generate_DF_comm(df_comb['A_t_CWSH2'], 'A_CWSH2')
df_A_t_PRESS1 = generate_DF_comm(df_comb['A_t_PRESS1'], 'A_t_PRESS1')
df_A_t_PRESS2 = generate_DF_comm(df_comb['A_t_PRESS2'], 'A_t_PRESS2')
df_PU26A_OP = generate_DF_comm(df_comb['PU26A_OP'], 'A_PU26A_OP')
df_PU27A_OP = generate_DF_comm(df_comb['PU27A_OP'], 'A_PU27A_OP')
df_106PU26A_CUR = generate_DF_comm(df_comb['106PU26A_CUR'], 'A_PU26A_CUR')
df_106PU27A_CUR = generate_DF_comm(df_comb['106PU27A_CUR'], 'A_PU27A_CUR')
#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())

dfn = df_A_CWP_1.join(df_A_CWP_2, how='outer')
dfn1 = df_A_N2DRY.join(df_A_MANFP, how='outer')
dfn2 = df_A_t_FEED.join(df_A_t_CWSH1, how='outer')
dfn3 = df_A_t_CWSH2.join(df_A_t_PRESS1, how='outer')
dfn4 = df_A_t_PRESS2.join(df_PU26A_OP, how='outer')
dfn5 = df_PU27A_OP.join(df_106PU26A_CUR, how='outer')
dfn6 = df_106PU27A_CUR.join(dfn, how='outer')
dfn7 = dfn1.join(dfn2, how='outer')
dfn8 = dfn3.join(dfn4, how='outer')
dfn9 = dfn5.join(dfn6, how='outer')
dfn10 = dfn7.join(dfn8, how='outer')
dfn11 = dfn9.join(dfn10, how='outer')

df_PressureFilter_Param_A = dfn11

#-----
print('Done. Generated the merged dataframes of PiData parameters for pressure filter alpha.', datetime.datetime.now())


Done
Done. Filtration Cycle Time generated. 2020-07-29 18:49:12.075911
2020-04-01 16:49:00
2020-04-01 17:08:00
2020-04-01 17:30:00
2020-04-01 17:49:00
2020-04-01 18:07:00
2020-04-01 18:25:00
2020-04-01 18:43:00
2020-04-01 19:00:00
2020-04-01 19:18:00
2020-04-01 19:37:00
2020-04-01 19:54:00
2020-04-01 20:12:00
2020-04-01 20:30:00
2020-04-01 20:48:00
2020-04-01 21:06:00
2020-04-01 21:24:00
2020-04-01 21:42:00
2020-04-01 22:00:00
2020-04-01 22:19:00
2020-04-01 22:38:00
2020-04-01 22:56:00
2020-04-02 00:52:00
2020-04-02 01:26:00
2020-04-02 02:05:00
2020-04-02 02:25:00
2020-04-02 02:46:00
2020-04-02 03:07:00
2020-04-02 03:30:00
2020-04-02 10:17:00
2020-04-02 10:49:00
2020-04-02 12:36:00
2020-04-02 13:03:00
2020-04-02 13:24:00
2020-04-02 13:45:00
2020-04-02 14:06:00
2020-04-02 14:28:00
2020-04-02 14:51:00
2020-04-02 17:38:00
2020-04-02 23:48:00
2020-04-16 01:30:00
2020-04-16 15:15:00
2020-04-16 15:50:00
2020-04-16 17:16:00
2020-04-16 18:44:00
2020-04-16 19:11:00
2020-04-16 19:38:00
2020-04-

Done. Generated dataframes of each parameter for pressure filter alpha. 2020-07-29 18:49:13.950950
Done. Generated the merged dataframes of PiData parameters for pressure filter alpha. 2020-07-29 18:49:13.958929


# SELECT THE BRAVO PARAMETERS

In [56]:
#Get the local min, max. Then generate the keys
ft_cyc_limB = getTheLocalMinMax(df_comb['B_t_FEED'], df_comb['B_t_DRY'])
keyslistB = list(ft_cyc_limB.keys())
#------
print('Done. Filtration Cycle Time generated.', datetime.datetime.now())
#Generate the dataframes for B
df_B_CWP_1 = generate_dfB_CW1(df_comb['FT02B_CWP'], 'B_CWP_1')
df_B_CWP_2 = generate_dfB_CW2(df_comb['FT02B_CWP'], 'B_CWP_2')
df_B_N2DRY = generate_dfB_for_N2DRY(df_comb['B_t_DRY'], 'B_N2DRY')
df_B_MANFP = generate_dfB_comm(df_comb['FT02B_MANFP'], 'B_MANF_P')
df_B_t_FEED = generate_dfB_comm(df_comb['B_t_FEED'], 'B_FILT_T')
df_B_t_CWSH1 = generate_dfB_comm(df_comb['B_t_CWSH1'], 'B_CWSH1')
df_B_t_CWSH2 = generate_dfB_comm(df_comb['B_t_CWSH2'], 'B_CWSH2')
df_B_t_PRESS1 = generate_dfB_comm(df_comb['B_t_PRESS1'], 'B_t_PRESS1')
df_B_t_PRESS2 = generate_dfB_comm(df_comb['B_t_PRESS2'], 'B_t_PRESS2')
df_PU26B_OP = generate_dfB_comm(df_comb['PU26B_OP'], 'B_PU26B_OP')
df_PU27B_OP = generate_dfB_comm(df_comb['PU27B_OP'], 'B_PU27B_OP')
df_106PU26B_CUR = generate_dfB_comm(df_comb['106PU26B_CUR'], 'B_PU26B_CUR')
df_106PU27B_CUR = generate_dfB_comm(df_comb['106PU27B_CUR'], 'B_PU27B_CUR')

#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())
#Combining the dataframes for bravo parameters
dfnB = df_B_CWP_1.join(df_B_CWP_2, how='outer')
dfn1B = df_B_N2DRY.join(df_B_MANFP, how='outer')
dfn2B = df_B_t_FEED.join(df_B_t_CWSH1, how='outer')
dfn3B = df_B_t_CWSH2.join(df_B_t_PRESS1, how='outer')
dfn4B = df_B_t_PRESS2.join(df_PU26B_OP, how='outer')
dfn5B = df_PU27B_OP.join(df_106PU26B_CUR, how='outer')
dfn6B = df_106PU27B_CUR.join(dfnB, how='outer')
dfn7B = dfn1B.join(dfn2B, how='outer')
dfn8B = dfn3B.join(dfn4B, how='outer')
dfn9B = dfn5B.join(dfn6B, how='outer')
dfn10B = dfn7B.join(dfn8B, how='outer')
dfn11B = dfn9B.join(dfn10B, how='outer')
df_PressureFilter_Param_B = dfn11B
#-----
print('Done. Generated dataframes of each parameter for pressure filter alpha.', datetime.datetime.now())


Done
Done. Filtration Cycle Time generated. 2020-07-29 18:49:14.003781
2020-04-01 17:28:00
2020-04-01 17:49:00
2020-04-01 18:10:00
2020-04-01 18:30:00
2020-04-01 18:51:00
2020-04-01 19:11:00
2020-04-01 19:31:00
2020-04-01 19:51:00
2020-04-01 20:12:00
2020-04-01 20:32:00
2020-04-01 20:52:00
2020-04-01 21:12:00
2020-04-01 21:32:00
2020-04-01 21:53:00
2020-04-01 22:16:00
2020-04-01 22:37:00
2020-04-01 22:57:00
2020-04-01 23:18:00
2020-04-02 10:00:00
2020-04-02 10:27:00
2020-04-02 11:42:00
2020-04-02 13:24:00
2020-04-02 15:42:00
2020-04-02 16:17:00
2020-04-02 19:04:00
2020-04-02 20:17:00
2020-04-17 02:32:00
2020-04-17 03:04:00
2020-04-17 03:37:00
2020-04-17 04:49:00
2020-04-17 05:42:00
2020-04-17 06:10:00
2020-04-17 06:43:00
2020-04-17 07:13:00
2020-04-17 07:47:00
2020-04-17 08:20:00
2020-04-17 08:52:00
2020-04-17 09:27:00
2020-04-17 10:02:00
2020-04-17 11:44:00
2020-04-17 12:19:00
2020-04-17 12:55:00
2020-04-17 13:28:00
2020-04-17 14:03:00
2020-04-17 14:37:00
2020-04-17 15:09:00
2020-04-

Done. Generated dataframes of each parameter for pressure filter alpha. 2020-07-29 18:49:14.875830
Done. Generated dataframes of each parameter for pressure filter alpha. 2020-07-29 18:49:14.886135


In [57]:
df_PressureFilter_Param_A.to_excel(r'C:\Users\v.t.flores\Documents\PressureFilter_cycleData_042020_A.xlsx')
df_PressureFilter_Param_B.to_excel(r'C:\Users\v.t.flores\Documents\PressureFilter_cycleData_042020_B.xlsx')